In [4]:
import gc
import os
import pickle
import random
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as APS
import polars as pl

import gc
import os
import pickle
import random
import joblib
import pandas as pd
# import polars as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
import numpy as np


In [5]:
class Config:
    PREPROCESS = False
    KAGGLE_NOTEBOOK = False
    DEBUG = True
    
    SEED = 42
    EPOCHS = 5
    BATCH_SIZE = 4096
    LR = 1e-3
    WD = 0.05
    PATIENCE = 5
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    NBR_FOLDS = 15
    SELECTED_FOLDS = [0]
    
    

In [6]:
if Config.KAGGLE_NOTEBOOK:
    RAW_DIR = "/kaggle/input/leash-BELKA/"
    PROCESSED_DIR = "/kaggle/input/belka-enc-dataset"
    OUTPUT_DIR = ""
    MODEL_DIR = ""
else:
    RAW_DIR = "../data/raw/"
    PROCESSED_DIR = "../data/processed/"
    OUTPUT_DIR = "../data/result/"
    MODEL_DIR = "../models/"

In [8]:
train = pl.read_parquet(os.path.join(RAW_DIR, "train.parquet"), n_rows=10000)
test = pl.read_parquet(os.path.join(RAW_DIR, "test.parquet"), n_rows=10000)

In [13]:
# 先頭のみ表示
train.head()

id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
i64,str,str,str,str,str,i64
0,"""C#CC[C@@H](CC(=O)O)NC(=O)OCC1c…","""C#CCOc1ccc(CN)cc1.Cl""","""Br.Br.NCC1CCCN1c1cccnn1""","""C#CCOc1ccc(CNc2nc(NCC3CCCN3c3c…","""BRD4""",0
1,"""C#CC[C@@H](CC(=O)O)NC(=O)OCC1c…","""C#CCOc1ccc(CN)cc1.Cl""","""Br.Br.NCC1CCCN1c1cccnn1""","""C#CCOc1ccc(CNc2nc(NCC3CCCN3c3c…","""HSA""",0
2,"""C#CC[C@@H](CC(=O)O)NC(=O)OCC1c…","""C#CCOc1ccc(CN)cc1.Cl""","""Br.Br.NCC1CCCN1c1cccnn1""","""C#CCOc1ccc(CNc2nc(NCC3CCCN3c3c…","""sEH""",0
3,"""C#CC[C@@H](CC(=O)O)NC(=O)OCC1c…","""C#CCOc1ccc(CN)cc1.Cl""","""Br.NCc1cccc(Br)n1""","""C#CCOc1ccc(CNc2nc(NCc3cccc(Br)…","""BRD4""",0
4,"""C#CC[C@@H](CC(=O)O)NC(=O)OCC1c…","""C#CCOc1ccc(CN)cc1.Cl""","""Br.NCc1cccc(Br)n1""","""C#CCOc1ccc(CNc2nc(NCc3cccc(Br)…","""HSA""",0


In [14]:
test.head()

id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name
i64,str,str,str,str,str
295246830,"""C#CCCC[C@H](NC(=O)OCC1c2ccccc2…","""C=Cc1ccc(N)cc1""","""C=Cc1ccc(N)cc1""","""C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)c…","""BRD4"""
295246831,"""C#CCCC[C@H](NC(=O)OCC1c2ccccc2…","""C=Cc1ccc(N)cc1""","""C=Cc1ccc(N)cc1""","""C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)c…","""HSA"""
295246832,"""C#CCCC[C@H](NC(=O)OCC1c2ccccc2…","""C=Cc1ccc(N)cc1""","""C=Cc1ccc(N)cc1""","""C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)c…","""sEH"""
295246833,"""C#CCCC[C@H](NC(=O)OCC1c2ccccc2…","""C=Cc1ccc(N)cc1""","""CC(O)Cn1cnc2c(N)ncnc21""","""C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)c…","""BRD4"""
295246834,"""C#CCCC[C@H](NC(=O)OCC1c2ccccc2…","""C=Cc1ccc(N)cc1""","""CC(O)Cn1cnc2c(N)ncnc21""","""C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)c…","""HSA"""
